In [6]:
# ================================================================
# PHASE 1: Download BTC & ETH Historical Data (Binance Spot Only)
# ================================================================

import os
import time
import pandas as pd
import ccxt

# -----------------------------
# 1. Setup directories
# -----------------------------
DATA_DIR = os.path.join("..", "data_raw")
os.makedirs(DATA_DIR, exist_ok=True)

# -----------------------------
# 2. Binance Spot-only Connection (Safe Mode)
# -----------------------------
# =========================================================
# Binance SPOT-ONLY Connection (Correct Final Version)
# =========================================================
import ccxt

print("📡 Connecting to Binance Spot API...")

exchange = ccxt.binance({
    "enableRateLimit": True,
    "timeout": 60000,  # 60-second timeout
    "options": {
        "defaultType": "spot",
        "adjustForTimeDifference": True,
        "warnOnFetchOHLCVLimitArgument": False,
    }
})

# Manually disable futures endpoints
for key in ["fapiPublic", "fapiPrivate", "dapiPublic", "dapiPrivate", "eapiPublic", "eapiPrivate"]:
    if key in exchange.urls:
        exchange.urls[key] = None

exchange.load_markets(reload=True)
print("✅ Connected successfully to Binance SPOT markets.\n")

# -----------------------------
# 3. Helper functions
# -----------------------------
def timeframe_to_ms(tf):
    ms = {
        "1m": 60_000, "3m": 180_000, "5m": 300_000, "15m": 900_000,
        "30m": 1_800_000, "1h": 3_600_000, "4h": 14_400_000, "1d": 86_400_000
    }
    return ms[tf]


def fetch_ohlcv_all(symbol="BTC/USDT", timeframe="1h", since_iso="2021-01-01T00:00:00Z"):
    since = exchange.parse8601(since_iso)
    tf_ms = timeframe_to_ms(timeframe)
    all_rows = []
    limit = 1000

    while True:
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)
        except ccxt.RequestTimeout:
            print("⚠️  Timeout – retrying in 10 s...")
            time.sleep(10)
            continue
        except Exception as e:
            print(f"❌ Error fetching batch: {e}")
            break

        if not ohlcv:
            break

        all_rows += ohlcv
        since = ohlcv[-1][0] + tf_ms
        time.sleep(exchange.rateLimit / 1000)

        if since > exchange.milliseconds() - tf_ms:
            break

    df = pd.DataFrame(all_rows, columns=["time", "open", "high", "low", "close", "volume"])
    df["time"] = pd.to_datetime(df["time"], unit="ms", utc=True).dt.tz_convert(None)
    for c in ["open", "high", "low", "close", "volume"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna().drop_duplicates(subset=["time"]).reset_index(drop=True)
    return df


def fetch_ohlcv_yearwise(symbol, timeframe, start_year=2021):
    all_data = []
    current_year = start_year
    this_year = time.gmtime().tm_year
    while current_year <= this_year:
        since_iso = f"{current_year}-01-01T00:00:00Z"
        print(f"📅 Fetching {symbol} {timeframe} for {current_year} ...")
        df = fetch_ohlcv_all(symbol, timeframe, since_iso)
        all_data.append(df)
        current_year += 1
    return pd.concat(all_data, ignore_index=True)


def save_symbol(symbol, timeframe, start_year, file_tag):
    print(f"\n🔹 Downloading {symbol} {timeframe} from {start_year} ...")
    df = fetch_ohlcv_yearwise(symbol, timeframe, start_year)
    fn = f"{file_tag}{symbol.replace('/','')}_{timeframe}.csv"
    path = os.path.join(DATA_DIR, fn)
    df.to_csv(path, index=False)
    print(f"✅ Saved: {path}  | Rows: {len(df)}")
    return df

# -----------------------------
# 4. Download loop with retries
# -----------------------------
symbols_timeframes = [
    ("BTC/USDT", "1h"),
    ("ETH/USDT", "1h"),
    ("BTC/USDT", "15m"),
    ("ETH/USDT", "15m"),
]

for symbol, tf in symbols_timeframes:
    for attempt in range(3):
        try:
            save_symbol(symbol, tf, 2021, "raw_")
            break
        except ccxt.RequestTimeout:
            print(f"⚠️ Timeout fetching {symbol} {tf}, retry {attempt+1}/3 in 10 s...")
            time.sleep(10)
        except Exception as e:
            print(f"❌ Failed to download {symbol} {tf}: {e}")
            break

print("\n🎉 All downloads completed! Check your data_raw/ folder.")


📡 Connecting to Binance Spot API...
✅ Connected successfully to Binance SPOT markets.


🔹 Downloading BTC/USDT 1h from 2021 ...
📅 Fetching BTC/USDT 1h for 2021 ...
📅 Fetching BTC/USDT 1h for 2022 ...
📅 Fetching BTC/USDT 1h for 2023 ...
📅 Fetching BTC/USDT 1h for 2024 ...
📅 Fetching BTC/USDT 1h for 2025 ...
✅ Saved: ..\data_raw\raw_BTCUSDT_1h.csv  | Rows: 125195

🔹 Downloading ETH/USDT 1h from 2021 ...
📅 Fetching ETH/USDT 1h for 2021 ...
📅 Fetching ETH/USDT 1h for 2022 ...
📅 Fetching ETH/USDT 1h for 2023 ...
📅 Fetching ETH/USDT 1h for 2024 ...
📅 Fetching ETH/USDT 1h for 2025 ...
✅ Saved: ..\data_raw\raw_ETHUSDT_1h.csv  | Rows: 125195

🔹 Downloading BTC/USDT 15m from 2021 ...
📅 Fetching BTC/USDT 15m for 2021 ...
📅 Fetching BTC/USDT 15m for 2022 ...
📅 Fetching BTC/USDT 15m for 2023 ...
📅 Fetching BTC/USDT 15m for 2024 ...
📅 Fetching BTC/USDT 15m for 2025 ...
✅ Saved: ..\data_raw\raw_BTCUSDT_15m.csv  | Rows: 500754

🔹 Downloading ETH/USDT 15m from 2021 ...
📅 Fetching ETH/USDT 15m for 2021 